# Student Loan Risk with Deep Learning

In [ ]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [ ]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

In [ ]:
# Review the data types associated with the columns
loans_df.dtypes

In [ ]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [ ]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]


# Display a sample of y
y[:5]


In [ ]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.copy()


# Review the features DataFrame
X = X.drop("credit_ranking", axis=1)
X.head()


### Step 3: Split the features and target sets into training and testing datasets.


In [ ]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()


# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)


# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [ ]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features


In [ ]:
# Define the number of hidden nodes for the first hidden layer
first_hidden_nodes = 8

# Define the number of hidden nodes for the second hidden layer
second_hidden_nodes = 5

# Define the number of neurons in the output layer
number_output_neurons = 1


In [ ]:
# Create the Sequential model instance
nn = Sequential()

# Add the first hidden layer
nn.add(Dense(units=first_hidden_nodes, input_dim=number_input_features, activation="relu"))

# Add the second hidden layer
nn.add(Dense(units=second_hidden_nodes, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))


In [ ]:
# Display the Sequential model summary
nn.summary()


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [ ]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
# Fit the model using 50 epochs and the training data
model = nn.fit(X_train_scaled, y_train, epochs=50)


### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [ ]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Export your model to a keras file
nn.save(str(file_path))


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [ ]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Load the model to a new object
nn_imported = tf.keras.models.load_model(str(file_path))


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [ ]:
# Make predictions with the test data
y_pred = nn_imported.predict(X_test)

# Display a sample of the predictions
y_pred[:5]


In [ ]:
# Save the predictions to a DataFrame and round the predictions to binary results
results = pd.DataFrame(y_test)
results["Predicted"] = y_pred
results["Predicted"] = results["Predicted"].apply(lambda x: 1 if x >= 0.5 else 0)
results.head()


### Step 4: Display a classification report with the y test data and predictions

In [ ]:
# Print the classification report with the y test data and predictions
print(classification_report(results["credit_ranking"], results["Predicted"]))


---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

For the answer to this question, look no further than the banks.  When you request a loan they collect:
- Your Personal data: name, age, location, employment, marital status etc. 
- Academic Information: Since we are dealing specifically with student loans we should consider grades, academic program, family support and potential future income.
- Financial Information:
credit score, loan eligibility, current debt level, ability to repay on time are all common data points.
- Details of the Loan:
interest rates, terms, potential co-signers, repayment history etc.

All of these provide a clearer picture of the lender-borrower relation ship and its potential success.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

Based on my answer above, I would say that the Context based filtering would be the way to go.  That said though, most students do NOT have much evidence of payment history or credit score because most teens do not get loans nor credit cards while they are living under their parent's roof. With this in mind, it may be worth trying a collaborative filtering model that is based on whether the applicants shared matching data with the past best payers.  

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

There are so many things to consider here. Ethics is a big issue. Do your data and filter choices give everyone a fair shake? Are you unfairly profiling people? Is the collaborative filter model I mentioned above even ethical?  Is it a privacy violation? All data use would necessarily have to follow the current laws and regulations. I would also consider the shortcomings of the provided data potential problem. No system is perfect. For instance, the credit rating system is considered to be accurate though it does not consider where you get the money to pay your debt, only that you pay and do so on time. You could be temporarily using severence or inheritance to pay off your credit cards, artificially raising your credit score for a while. Creditors can be misled by this to extend loans or cards when you do not have a steady job or ongoing income. Another issue is changing situations. Tuition increases requiring bigger loans, interest rates rise and fall sometimes favoring the lender, sometimes the borrower. economic conditions such as recessions and booms also change the equation for loan terms and volume of borrowers shopping for them.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

